All the necessary imports

In [115]:
import spacy
import pandas as pd
import os
from pathlib import Path
from spacy.matcher import Matcher
from collections import Counter

In [95]:
# Load Serbian language model
nlp = spacy.load('hr_core_news_lg')

In [96]:
def read_letters(folder_path):
    """
    Reads all letters from the specified folder
    Returns a DataFrame with filename and content
    """
    letters = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                letters.append({
                    'filename': filename,
                    'content': content
                })

    print(f"Loaded {len(letters)} letters.")
    
    return pd.DataFrame(letters)

In [97]:
letters_df = read_letters("letters")

Loaded 12 letters.


### Task 2:
Splitting the letters into sentences and performing the analysis.

In [98]:
def analyze_sentences(df):
    """
    Analyzes sentences in each letter, focusing on length and complexity
    Returns DataFrame with detailed sentence analysis
    """
    sentence_analysis = []
    
    for _, row in df.iterrows():
        doc = nlp(row['content'])
        
        # Analyze each sentence
        for sent in doc.sents:
            # Count words (excluding punctuation)
            word_count = len([token for token in sent if not token.is_punct])
            
            # Calculate average word length (excluding punctuation)
            avg_word_length = sum(len(token.text) for token in sent if not token.is_punct) / word_count if word_count > 0 else 0
            
            sentence_analysis.append({
                'filename': row['filename'],
                'sentence': sent.text,
                'word_count': word_count,
                'avg_word_length': round(avg_word_length, 2),
                'is_long': word_count > 20  # Flag for long sentences
            })
    
    return pd.DataFrame(sentence_analysis)

def get_sentence_statistics(sentence_df):
    """
    Calculates statistical summary of sentence analysis
    """
    stats = {
        'total_sentences': len(sentence_df),
        'avg_sentence_length': sentence_df['word_count'].mean(),
        'long_sentences_count': len(sentence_df[sentence_df['is_long']]),
        'max_sentence_length': sentence_df['word_count'].max(),
        'min_sentence_length': sentence_df['word_count'].min()
    }
    return stats


def analyze_letters_sentences(letters_df):
    """
    Main function to analyze all letters and their sentences
    """
    # Analyze sentences
    sentence_analysis = analyze_sentences(letters_df)
    
    # Get statistics
    stats = get_sentence_statistics(sentence_analysis)
    
    # Get notable sentences (longer than 20 words)
    long_sentences = sentence_analysis[sentence_analysis['is_long']].sort_values('word_count', ascending=False)
    
    return letters_df, sentence_analysis, stats, long_sentences

def print_analysis_summary(stats, long_sentences):
    print("\nSentence Analysis in Andrić's Letters:")
    print("=====================================")
    print(f"Total number of analyzed sentences: {stats['total_sentences']}")
    print(f"Average sentence length: {stats['avg_sentence_length']:.2f} words")
    print(f"Number of long sentences (>20 words): {stats['long_sentences_count']}")
    print(f"Longest sentence has {stats['max_sentence_length']} words")
    print("=====================================")
    
    len_long_sentences = stats['long_sentences_count']
    # check whether long sentences is not empty
    if len_long_sentences != 0:
        # print first min between len of long_sentences and 3
        print("\nExamples of the longest sentences:")
        print("=====================================")

        for _, row in long_sentences.head(min(len_long_sentences, 7)).iterrows():
            print(f"Letter: {row['filename']}")
            print(f"Sentence: {row['sentence']}")
            print(f"Word count: {row['word_count']}")
            print(f"Avg. word length: {row['avg_word_length']}")
            print()



Andrić writes relatively short sentences, there are only around 15% of sentences that have more than 20 words. The average sentence length is 12 words. His writting style is simplistic and easy to read, there are not many unnecessary adjectives in them.

In [99]:
letters_df, sentence_analysis, stats, long_sentences = analyze_letters_sentences(letters_df)
sentences = sentence_analysis['sentence']
print_analysis_summary(stats, long_sentences)


Sentence Analysis in Andrić's Letters:
Total number of analyzed sentences: 83
Average sentence length: 11.58 words
Number of long sentences (>20 words): 12
Longest sentence has 50 words

Examples of the longest sentences:
Letter: letter_2.txt
Sentence: U tome, kako treba dočekati bolest i podnositi Ьоl,
ја bih mogao biti Vaš učenik i primati Vaše savete а ne
Vi moje.
Sad, kad је sve па dobrom putu, nadam se da ćemo
se još ovoga leta videti i porazgovarati, i radujem se
unapred.
Word count: 50
Avg. word length: 3.66

Letter: letter_2.txt
Sentence: Gledaću svakako da dođem па jedan dan u
Zagreb, kad budem polazio па odmor; а dotle, držite se
dobro i budite tvrdi kao uvek, dragi, stari mој Tugomire.
Grli Vas i pozdravlja
Vaš
Ivo

Word count: 40
Avg. word length: 3.73

Letter: letter_5.txt
Sentence: ,
šaljem vam u prilogu 50 franaka i molim vas da
budete ljubazni i da mi pošaljete Antologiju Lirike "Misli"
(poslednje izdanje, sa slikаmа) i Antologiju Ljubavnе Lirike 
od g. Bož. Kovačevića

### Task 3:
Tokenizing the words and removing the stopwords.

In [100]:
def tokenize(sentences):
    """
    Tokenize the input sentences
    
    Parameters:
    sentences (list): List of sentences to tokenize
    
    Returns:
    list: List of lists where each inner list contains tokens for one sentence
    """
    tokenized_sentences = []
    
    for sentence in sentences:
        doc = nlp(sentence)
        # Get all tokens that aren't punctuation or whitespace
        tokens = [token.text.lower() for token in doc 
                if not token.is_punct and not token.is_space]
        tokenized_sentences.append(tokens)
    
    return tokenized_sentences

def remove_stop_words(sentences):
    """
    Remove stop words from tokenized sentences
    
    Parameters:
    sentences (list): List of lists containing tokenized sentences
    
    Returns:
    list: List of lists with stop words removed
    """
    cleaned_sentences = []
    
    for sentence in sentences:
        # Process each token to check if it's a stop word
        cleaned_tokens = [token for token in sentence 
                        if not nlp.vocab[token].is_stop]
        cleaned_sentences.append(cleaned_tokens)
    
    return cleaned_sentences

In [101]:
tokenized_sentences = tokenize(sentences)
cleaned_sentences = remove_stop_words(tokenized_sentences)
print("Tokenized sentence: \n")
print(tokenized_sentences[3])
print("##############################################")
print("After removing stop words: \n")
print(cleaned_sentences[3])

Tokenized sentence: 

['ја', 'sam', 'proveo', 'vrlo', 'rđavu', 'zimu', 'јеr', 'sam', 'dva', 'puta', 'рrеbоlео', 'grip', 'i', 'sada', 'vučem', 'posledice']
##############################################
After removing stop words: 

['ја', 'proveo', 'rđavu', 'zimu', 'јеr', 'dva', 'puta', 'рrеbоlео', 'grip', 'sada', 'vučem', 'posledice']


### Task 4:
Stemming vs Lemmatization
Couldn't find library which supports stemming for Serbian language, so I used only lemmatization. Also lemmatization should be more suitable for this task, as Serbian language is really complex and has a lot of exceptions.

In [106]:
def perform_lemmatization(sentences):
    """
    Perform lemmatization on sentences
    
    Parameters:
    sentences (list): List of sentences
    
    Returns:
    list: List of lists containing lemmatized tokens
    """
    lemmatized_sentences = []
    
    for sentence in sentences:
        doc = nlp(sentence)
        lemmas = [token.lemma_ for token in doc 
                if not token.is_punct and not token.is_space]
        lemmatized_sentences.append(lemmas)
    
    return lemmatized_sentences

def compare_lemmatized_sentences(original_sentences, lemmatized_sentences):
    """
    Compare original and lemmatized sentences
    
    Parameters:
    original_sentences (list): List of original sentences
    lemmatized_sentences (list): List of lemmatized tokens
    """
    for i in range(len(original_sentences)):
        print(f"{i} sentence")
        print(f"\nOriginal: {original_sentences[i]}")
        print(f"Lemmatized: {' '.join(lemmatized_sentences[i])}")
        print("##############################################")



def compare_words_and_lemmas(sentences):
    """
    Compare original tokens with their lemmatized forms
    
    Parameters:
    sentences (list): List of sentences
    
    Returns:
    list: List of tuples containing (original_token, lemma)
    """
    token_lemma_pairs = []
    
    for sentence in sentences:
        doc = nlp(sentence)
        # Get pairs of tokens and their lemmas
        pairs = [(token.text, token.lemma_) for token in doc 
                if not token.is_punct and not token.is_space]
        token_lemma_pairs.extend(pairs)
    
    return token_lemma_pairs


In [105]:
examples = [2, 3, 46, 75]
example_sentences = [sentences[i] for i in examples]
lemmatized_sentences = perform_lemmatization(example_sentences)
compare_lemmatized_sentences(example_sentences, lemmatized_sentences)


0 sentence

Original: ,
šaljem vam u prilogu 50 franaka i molim vas da
budete ljubazni i da mi pošaljete Antologiju Lirike "Misli"
(poslednje izdanje, sa slikаmа) i Antologiju Ljubavnе Lirike 
od g. Bož. Kovačevića.

Lemmatized: šaljem vi u prilog 50 franak i moliti vi da budete ljubazan i da mi pošaljete Antologija Lirika misli poslednji izdanje sa slikаmа i Antologiju Ljubavnе Lirika od g. Bož Kovačević
##############################################
1 sentence

Original: Ја sam proveo vrlo rđavu zimu, јеr sam dva puta
рrеbоlео grip, i sada vučem posledice.
Lemmatized: Ја biti provesti vrlo rđava zima јеr biti dva put рrеbоlео grip i sada vučem posledica
##############################################
2 sentence

Original: Gledaću svakako da dođem па jedan dan u
Zagreb, kad budem polazio па odmor; а dotle, držite se
dobro i budite tvrdi kao uvek, dragi, stari mој Tugomire.
Grli Vas i pozdravlja
Vaš
Ivo

Lemmatized: Gledaću svakako da dođem па jedan dan u Zagreb kad budem polaziti па od

Word Pattern Analysis:

1. Prepositions frequency: There is a notable presence of prepositions in the text, which suggests complex sentence structures and detailed descriptions of relationships between objects, people, and concepts.

Emotional Tone Dichotomy:
1. Positive language when referring to others:
- The words used to describe or address other people tend to be gentle and endearing
- This suggests a respectful and warm approach in his interpersonal communications

In [114]:
# Get word-lemma pairs for comparison
word_lemma_pairs = compare_words_and_lemmas(sentences)

print("Word-Lemma pairs: \n")
cnt = 0
for word, lemma in word_lemma_pairs:
    if cnt > 70:
        break
    if word != lemma:
        print(f"Word: {word}, Lemma: {lemma}")
        cnt += 1

Word-Lemma pairs: 

Word: Dragi, Lemma: dragi
Word: Cvijanoviću, Lemma: Cvijanović
Word: vam, Lemma: vi
Word: prilogu, Lemma: prilog
Word: franaka, Lemma: franak
Word: molim, Lemma: moliti
Word: vas, Lemma: vi
Word: ljubazni, Lemma: ljubazan
Word: Antologiju, Lemma: Antologija
Word: Lirike, Lemma: Lirika
Word: Misli, Lemma: misli
Word: poslednje, Lemma: poslednji
Word: Lirike, Lemma: Lirika
Word: Kovačevića, Lemma: Kovačević
Word: sam, Lemma: biti
Word: proveo, Lemma: provesti
Word: rđavu, Lemma: rđava
Word: zimu, Lemma: zima
Word: sam, Lemma: biti
Word: puta, Lemma: put
Word: posledice, Lemma: posledica
Word: Nadam, Lemma: nadati
Word: se, Lemma: sebe
Word: ste, Lemma: biti
Word: svi, Lemma: sav
Word: vaši, Lemma: vaš
Word: Kako, Lemma: kako
Word: ide, Lemma: ići
Word: naša, Lemma: naš
Word: se, Lemma: sebe
Word: više, Lemma: mnogo
Word: Dragi, Lemma: dragi
Word: nemam, Lemma: nemati
Word: nikakvih, Lemma: nikakav
Word: vijesti, Lemma: vijest
Word: Vas, Lemma: vi
Word: Primio, Lemma: 

### Task 5:
Phrase matching and analysis

In [138]:
import re
# Function to clean text by removing unwanted characters
def clean_text(text):
    return re.sub(r'[\n.,!?;:\"\'()\-\]]', ' ', text)

# Function to extract repeated phrases from a list of sentences
def extract_repeated_phrases(sentences, min_length=2, max_length=4, top_k=10):
    matcher = Matcher(nlp.vocab)
    phrase_counts = Counter()
    
    for sentence in sentences:
        cleaned_sentence = clean_text(sentence)
        doc = nlp(cleaned_sentence)
        
        # Extract n-grams (phrases with min_length to max_length words)
        for i in range(len(doc) - min_length + 1):
            for j in range(min_length, max_length + 1):
                if i + j <= len(doc):
                    phrase = " ".join([token.text.lower() for token in doc[i:i + j]]).strip()
                    phrase_counts[phrase] += 1
    
    # Filter phrases that appear more than once and sort by frequency
    repeated_phrases = {phrase: count for phrase, count in phrase_counts.items() if count > 1}
    sorted_phrases = sorted(repeated_phrases.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_phrases[:top_k]


The three most significant phrases are:
1. Molim vas (12)
2. Pozdravlja vas (7)
3. Nadam se (4)

The first phrase reflects Andrić’s humility, grace, and gentleness in addressing his friends.

The third phrase is often used in the context of expressing hope that his friends are doing well and that they will meet soon, demonstrating his care and thoughtfulness.

In [137]:
top_phrases = extract_repeated_phrases(sentences, top_k=10)
print("\nTop repeated phrases:")
print("=====================")
print("\n".join([f"{phrase}: {count}" for phrase, count in top_phrases]))


Top repeated phrases:
молим вас: 8
поздравља вас: 7
иво андрић: 7
mој tugomire: 6
цвијановићу: 5
vaš ivo: 5
molim vas: 4
nadam se: 4
da ste: 4
tugomire: 4


### Task 6:
POS Tagging - Noun to Verb Ratio

In [163]:
def return_docs(sentences):
    docs = []  # store docs for visualization
    
    for sentence in sentences:
        doc = nlp(sentence)
        docs.append(doc)    
    return docs

def visualize_dependencies(doc):
    """
    Visualize dependencies for a single sentence
    """
    options = {
        'distance': 110,
        'compact': True,
        'color': 'purple',
        'bg': 'white',
        'font': 'Times'
    }
    
    return spacy.displacy.render(doc, style='dep', options=options, jupyter=True)


def get_pos_distribution(pos_tags):
    """
    Calculate distribution of POS tags
    """
    return Counter(pos_tags)

def analyze_noun_verb_ratio(pos_tags):
    """
    Calculate the ratio of nouns to verbs
    """
    nouns = sum(1 for tag in pos_tags if tag == 'NOUN')
    verbs = sum(1 for tag in pos_tags if tag == 'VERB')
    
    ratio = nouns/verbs if verbs > 0 else 0
    return nouns, verbs, ratio

In [165]:
all_pos_tags = []

# Collect all POS tags
for sentence in sentences:
    doc = nlp(sentence)
    for token in doc:
        if not token.is_punct and not token.is_space:
            all_pos_tags.append(token.pos_)

# Get and print POS distribution
pos_dist = get_pos_distribution(all_pos_tags)
print("POS Distribution:")
print("=================")
for pos, count in pos_dist.most_common():
    print(f"{pos}: {count}")

# Get and print noun-verb analysis
nouns, verbs, ratio = analyze_noun_verb_ratio(all_pos_tags)
print(f"\nNumber of nouns: {nouns}")
print(f"Number of verbs: {verbs}")
print(f"Noun-to-Verb ratio: {ratio:.2f}")

POS Distribution:
PROPN: 206
NOUN: 129
VERB: 84
ADJ: 69
X: 57
ADV: 56
PRON: 53
AUX: 36
CCONJ: 34
ADP: 30
SCONJ: 28
DET: 27
NUM: 20
PART: 12
PUNCT: 5
SPACE: 1

Number of nouns: 129
Number of verbs: 84
Noun-to-Verb ratio: 1.54


In [164]:
# Visualize dependencies for each sentence
docs = return_docs(sentences)
print("\nDependency Visualizations:")
for doc in docs[3:7]:
    visualize_dependencies(doc)


Dependency Visualizations:
